In [160]:
import cv2                 # working with, mainly resizing, images
import numpy as np         # dealing with arrays
import os                  # dealing with directories
from random import shuffle # mixing up or currently ordered data that might lead our network astray in training.
from tqdm import tqdm      # a nice pretty percentage bar for tasks
IMG_SIZE = 32
LR = 1e-3
import pandas as pd

from sklearn.ensemble import RandomForestClassifier,BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import xgboost as xgb
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import f1_score
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
import glob

### sync data


In [358]:

path = r'D:\downloads\קבצים לעבודת גמר\extraFiles\Training\all data\sync' 
all_files = glob.glob(path + "/*.csv")
li = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, )
    li.append(df)
    frame_sy = pd.concat(li, axis=0, ignore_index=True)    
      
    
      


In [360]:
frame_sy.groupby(' Hand Type').count()

,Time,Frame ID,# hands,Position X,Position Y,Position Z,Velocity X,Velocity Y,Velocity Z,Pitch,...,Yaw,Wrist Pos X,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength
Hand Type,,,,,,,,,,,,,,,,,,,,,
left,37639,37639,37639,37639,37639,37639,37639,37639,37639,37639,...,37639,37639,37639,37639,37639,37639,37639,37639,37639,37639
right,37639,37639,37639,37639,37639,37639,37639,37639,37639,37639,...,37639,37639,37639,37639,37639,37639,37639,37639,37639,37639


In [361]:
x=frame_sy['Time'].values[0]+7
x

434.231

In [362]:
indexNames = frame_sy[frame_sy['Time'] <= x].index
# Delete these row indexes from dataFrame
frame_sy.drop(indexNames , inplace=True)

In [363]:
frame_sy

,Time,Frame ID,Hand Type,# hands,Position X,Position Y,Position Z,Velocity X,Velocity Y,Velocity Z,...,Yaw,Wrist Pos X,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength
846,434.2387,139878,right,2,121.96880,230.3867,61.918910,7.82505,-222.59100,121.91490,...,-0.320716,143.0681,170.9626,92.10286,290.9513,-17.46554,156.27410,0.0,0.036538,0.0
847,434.2387,139878,left,2,-80.92312,284.2069,29.011070,84.47700,-45.59256,134.20850,...,0.624079,-107.3702,221.7536,46.12994,-254.8560,22.47225,40.89467,0.0,0.000000,0.0
848,434.2553,139880,right,2,121.61940,226.1235,64.109630,-32.32643,-248.08270,109.73410,...,-0.333641,143.3853,166.9691,94.34527,293.7114,-20.13464,156.68940,0.0,0.001488,0.0
849,434.2553,139880,left,2,-79.71033,282.4789,31.691100,55.75132,-119.51820,158.45370,...,0.653397,-106.8187,220.1253,48.09507,-258.4003,23.92692,43.43317,0.0,0.000000,0.0
850,434.2718,139882,right,2,121.03880,222.2049,65.984120,-28.86916,-210.96420,109.70620,...,-0.344350,143.3656,163.3614,96.39383,296.6688,-21.13594,159.25370,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76014,856.2631,67309,right,2,101.91230,212.0264,42.535680,79.95810,113.41030,190.77140,...,-0.574317,131.8590,149.6264,66.09853,283.6185,-58.03491,95.29868,0.0,0.000000,0.0
76015,856.2958,67311,left,2,-81.57359,183.5054,-7.185198,-63.24731,121.75270,62.95970,...,0.459446,-109.7250,130.7078,34.03360,-295.5957,-14.97817,140.02630,0.0,0.000000,0.0
76016,856.2958,67311,right,2,103.84750,214.7484,47.070820,61.11773,95.13531,155.74530,...,-0.612753,134.0416,151.5924,68.17693,288.9688,-54.13820,94.23959,0.0,0.000000,0.0
76017,856.3295,67314,left,2,-84.05124,187.8450,-5.172269,-57.17249,109.20360,58.48146,...,0.434273,-109.5167,131.6489,33.32945,-282.9649,-31.35462,135.07610,0.0,0.000000,0.0


In [364]:
# Set index on a Dataframe
frame_sy.set_index(" Hand Type", 
              inplace = True)
# Using the operator .loc[]
# to select multiple rows
df_lf_sync = frame_sy.loc[["left"]]
df_rg_sync = frame_sy.loc[["right"]] 

In [365]:
frame_sy.groupby(' Hand Type').count()

,Time,Frame ID,# hands,Position X,Position Y,Position Z,Velocity X,Velocity Y,Velocity Z,Pitch,...,Yaw,Wrist Pos X,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength
Hand Type,,,,,,,,,,,,,,,,,,,,,
left,32914,32914,32914,32914,32914,32914,32914,32914,32914,32914,...,32914,32914,32914,32914,32914,32914,32914,32914,32914,32914
right,32914,32914,32914,32914,32914,32914,32914,32914,32914,32914,...,32914,32914,32914,32914,32914,32914,32914,32914,32914,32914


In [366]:
df_rg_sync.shape

(32914, 21)

In [367]:
df_lf_sync.shape

(32914, 21)

In [353]:
df_rg_sync.columns

Index(['Time', ' Frame ID', ' # hands', ' Position X', ' Position Y',
       ' Position Z', ' Velocity X', ' Velocity Y', ' Velocity Z', ' Pitch',
       ' Roll', ' Yaw', ' Wrist Pos X', ' Wrist Pos Y', ' Wrist Pos Z',
       ' Elbow pos X', ' Elbow Pos Y', ' Elbow Pos Z', ' Grab Strenth',
       ' Grab Angle', ' Pinch Strength'],
      dtype='object')

In [379]:
df_sync = pd.merge(df_lf_sync,df_rg_sync,how = "left",)

In [380]:
df_sync

,Time,Frame ID,# hands,Position X,Position Y,Position Z,Velocity X,Velocity Y,Velocity Z,Pitch,...,Yaw,Wrist Pos X,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength
0,434.2387,139878,2,-80.92312,284.2069,29.011070,84.47700,-45.59256,134.20850,1.128240,...,0.624079,-107.3702,221.7536,46.12994,-254.8560,22.472250,40.89467,0.0,0.0,0.0
1,434.2553,139880,2,-79.71033,282.4789,31.691100,55.75132,-119.51820,158.45370,1.139780,...,0.653397,-106.8187,220.1253,48.09507,-258.4003,23.926920,43.43317,0.0,0.0,0.0
2,434.2718,139882,2,-78.84495,279.8847,35.125760,29.94153,-131.54950,221.05070,1.159768,...,0.683048,-106.3970,217.4241,50.26434,-262.3120,24.761910,42.24427,0.0,0.0,0.0
3,434.2884,139884,2,-78.32854,276.6696,39.055000,30.61737,-215.07260,227.43820,1.190504,...,0.709288,-105.9420,213.8663,52.21570,-271.2896,29.791380,35.81071,0.0,0.0,0.0
4,434.3050,139885,2,-77.92608,275.3798,40.843280,46.92449,-150.37140,208.49700,1.205251,...,0.731741,-105.8699,212.5583,53.02802,-275.5691,32.855700,32.98580,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32909,856.1965,67304,2,-77.62601,173.7391,-14.213500,-46.75000,99.50911,71.01207,0.629213,...,0.429763,-107.8500,127.8215,32.85250,-318.7630,20.751860,138.01250,0.0,0.0,0.0
32910,856.2291,67306,2,-78.43058,176.0170,-12.152660,-33.69026,98.24361,59.77054,0.667618,...,0.434664,-108.0805,128.1729,33.40391,-314.6744,14.308990,139.99090,0.0,0.0,0.0
32911,856.2631,67309,2,-79.84506,180.2403,-9.015577,-68.45120,136.37580,87.63277,0.725945,...,0.456099,-109.0018,129.5246,33.89856,-304.9600,-0.548136,142.01090,0.0,0.0,0.0
32912,856.2958,67311,2,-81.57359,183.5054,-7.185198,-63.24731,121.75270,62.95970,0.762549,...,0.459446,-109.7250,130.7078,34.03360,-295.5957,-14.978170,140.02630,0.0,0.0,0.0


### Spontan data

In [422]:

path = r'D:\downloads\קבצים לעבודת גמר\extraFiles\Training\all data\Spontan' 
all_files = glob.glob(path + "/*.csv")
li = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)
    frame_sp = pd.concat(li, axis=0, ignore_index=True)    
      

In [423]:
frame_sp

,Time,Frame ID,Hand Type,# hands,Position X,Position Y,Position Z,Velocity X,Velocity Y,Velocity Z,...,Yaw,Wrist Pos X,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength
0,335.9984,128556,right,2,128.63570,224.0294,-12.29588,-18.26243,-140.54450,-117.577900,...,-1.588251,177.5125,173.5658,-14.569630,276.8753,-33.68264,91.61740,0.0,1.250576,0.0
1,335.9984,128556,left,2,-93.24845,143.9423,-37.81597,-109.09820,240.38980,-67.629590,...,0.873458,-147.0455,109.9380,-5.326688,-329.8577,-34.41571,93.88171,0.0,0.014736,0.0
2,336.0150,128558,right,2,129.15550,221.9923,-15.00181,47.36464,-64.25022,-168.858500,...,-1.528183,177.7894,171.1952,-14.677850,276.0645,-33.15913,97.88171,0.0,1.069930,0.0
3,336.0150,128558,left,2,-94.86554,148.5542,-39.00158,-80.99012,278.60220,-66.793560,...,0.879703,-148.2022,113.0909,-7.375025,-326.9279,-39.97873,85.98000,0.0,0.000000,0.0
4,336.0315,128560,right,2,129.87070,220.7944,-18.38681,35.05422,-84.74242,-205.720300,...,-1.461710,178.3730,169.9159,-15.246240,276.1767,-30.72506,104.13920,0.0,0.916969,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76116,673.4894,52512,left,2,-81.66566,203.9826,24.04028,240.80950,-160.56200,-222.444200,...,0.845894,-121.2401,146.1889,44.914080,-263.3239,-65.38571,89.68940,0.0,1.299514,0.0
76117,673.5233,52516,right,2,158.51870,172.6876,-29.32913,62.95145,93.99306,-63.022420,...,-0.452287,200.4651,148.9561,25.427530,400.5782,31.01108,139.27960,0.0,0.448368,0.0
76118,673.5233,52516,left,2,-77.57621,198.9182,20.57973,127.00290,-218.65710,-80.403090,...,0.827028,-120.1531,144.5244,44.439020,-276.8819,-56.07000,90.44517,0.0,0.542512,0.0
76119,673.5560,52519,right,2,159.35720,175.7603,-30.16282,10.30027,118.67660,-7.947631,...,-0.432002,199.5587,149.5821,24.801710,390.8217,24.22654,145.63370,0.0,0.513919,0.0


In [424]:
x=frame_sp['Time'].values[0]+7
x

342.9984

In [425]:
indexNames = frame_sp[frame_sp['Time'] <= x].index
# Delete these row indexes from dataFrame
frame_sp.drop(indexNames , inplace=True)

In [426]:
indexNames = frame_sp[frame_sp[' # hands'] == 1].index
# Delete these row indexes from dataFrame
frame_sp.drop(indexNames , inplace=True)

In [427]:
frame_sp.shape

(65972, 22)

In [428]:
indexNames = frame_sp[frame_sp[' # hands'] <= 1].index
# Delete these row indexes from dataFrame
frame_sp.drop(indexNames , inplace=True)

In [435]:
df_Spontan = pd.merge(df_lf_Spontan,df_rg_Spontan, how = "left",)

In [431]:
df_Spontan

,Time,Frame ID,# hands,Position X,Position Y,Position Z,Velocity X,Velocity Y,Velocity Z,Pitch,...,Yaw,Wrist Pos X,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength
0,343.0061,129364,2,-102.28700,198.3166,-2.122641,-174.57210,379.14680,28.479910,1.423625,...,1.481006,-164.0542,164.5314,-1.686393,-322.2086,-30.71549,11.754400,0.000000,1.870325,0.000000
1,343.0227,129365,2,-103.87210,201.4944,-2.063620,-178.50250,357.85480,6.646493,1.429039,...,1.486744,-165.7898,168.1620,-2.054844,-318.4708,-31.61653,7.529989,0.000000,1.851572,0.000000
2,343.0392,129367,2,-106.36910,207.4667,-2.337110,-126.78780,318.52840,-12.872290,1.441357,...,1.496526,-168.4939,174.6985,-3.134404,-316.4541,-28.81729,-1.155143,0.000000,1.865896,0.000000
3,343.0558,129369,2,-107.95090,212.1324,-2.788956,-74.28078,244.48230,-30.771300,1.444497,...,1.499930,-170.1878,179.6781,-4.185746,-315.3597,-25.78214,-9.287997,0.000000,1.899355,0.000000
4,343.0723,129371,2,-109.54270,216.8003,-3.337280,-96.49666,269.54340,-37.723870,1.431969,...,1.491886,-171.4205,183.7144,-4.797314,-311.4874,-25.01130,-16.183130,0.000000,1.950082,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32981,673.4224,52505,2,-93.01819,221.5837,37.953460,93.66040,-705.10900,-58.953580,1.390923,...,1.025598,-122.3104,155.2745,46.468090,-248.2816,-59.02841,118.862500,0.705491,2.090956,0.902568
32982,673.4561,52509,2,-87.87977,207.0944,30.250840,161.30820,-332.78340,-239.552200,1.268201,...,0.884023,-121.6544,144.0002,45.220440,-262.4403,-65.83310,101.248100,0.000000,1.569239,0.000000
32983,673.4894,52512,2,-81.66566,203.9826,24.040280,240.80950,-160.56200,-222.444200,1.124147,...,0.845894,-121.2401,146.1889,44.914080,-263.3239,-65.38571,89.689400,0.000000,1.299514,0.000000
32984,673.5233,52516,2,-77.57621,198.9182,20.579730,127.00290,-218.65710,-80.403090,1.037299,...,0.827028,-120.1531,144.5244,44.439020,-276.8819,-56.07000,90.445170,0.000000,0.542512,0.000000


### Alone data

In [464]:
path = r'D:\downloads\קבצים לעבודת גמר\extraFiles\Training\all data\Alone' 
all_files = glob.glob(path + "/*.csv")
li = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)
    frame_alone = pd.concat(li, axis=0, ignore_index=True)    
      

In [465]:
 frame_alone

,Time,Frame ID,Hand Type,# hands,Position X,Position Y,Position Z,Velocity X,Velocity Y,Velocity Z,...,Yaw,Wrist Pos X,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength
0,518.8087,149620,left,1,-97.60160,164.1213,42.81255,104.258900,-491.673000,-55.427550,...,1.661706,-166.7207,164.8320,27.58547,-378.1662,65.89330,-69.90282,0.000000,0.741855,0.000000
1,518.8252,149622,left,1,-95.78062,157.6353,42.18084,148.968300,-351.738600,-36.324220,...,1.623222,-165.4907,155.6419,29.71037,-380.3233,52.92441,-55.71872,0.000000,0.827916,0.000000
2,518.8418,149623,left,1,-94.39360,154.6338,41.93171,161.695800,-349.911700,-29.042770,...,1.602898,-164.3658,151.5701,31.05059,-381.7444,48.55032,-47.27909,0.000000,0.920401,0.000000
3,518.8587,149625,left,1,-92.02553,150.6723,42.10119,117.347500,-187.178100,24.425980,...,1.567605,-162.2663,144.6214,33.94577,-383.4688,40.27151,-30.72728,0.000000,1.034603,0.000000
4,518.8749,149627,left,1,-89.31953,148.1826,42.67065,155.334400,-135.633700,35.870550,...,1.535920,-159.4586,138.6087,36.68195,-381.0906,29.07133,-17.01065,0.000000,1.166705,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38817,764.4578,59984,left,1,-125.81370,214.8894,20.59324,20.170870,19.997410,37.082170,...,0.409534,-146.6314,148.2971,44.40766,-258.0887,-77.52971,112.97060,0.116068,1.901876,0.000000
38818,764.4913,59986,left,1,-125.92320,215.6769,22.09264,-6.867068,21.957800,42.485630,...,0.399021,-146.1113,148.8045,45.68930,-253.6847,-80.13625,109.99670,0.134497,1.952747,0.000000
38819,764.5253,59988,left,1,-125.62170,215.7428,23.39750,17.454730,-3.217659,31.446690,...,0.379251,-144.8174,148.4588,46.66075,-249.6531,-83.33176,105.00430,0.120680,1.984825,0.000000
38820,764.5587,59989,left,1,-125.13520,215.6353,23.80248,25.567410,-5.648659,21.284490,...,0.368866,-143.8482,148.1024,46.72861,-248.2768,-84.26936,103.46830,0.097735,1.988376,0.000000


In [466]:
x = frame_alone['Time'].values[0]+7
x

525.8087

In [467]:
indexNames = frame_alone[frame_alone['Time'] <= x].index
# Delete these row indexes from dataFrame
frame_alone.drop(indexNames , inplace=True)

In [468]:
indexNames = frame_alone[frame_alone[' # hands'] == 2].index
# Delete these row indexes from dataFrame
frame_alone.drop(indexNames , inplace=True)

In [469]:
indexNames = frame_alone[frame_alone[' Hand Type'] == 'right'].index
# Delete these row indexes from dataFrame
frame_alone.drop(indexNames , inplace=True)

In [470]:
frame_alone.shape

(22325, 22)

### right data

In [504]:
path = r'D:\downloads\קבצים לעבודת גמר\extraFiles\Training\all data\right' 
all_files = glob.glob(path + "/*.csv")
li = []
for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)
    df_right = pd.concat(li, axis=0, ignore_index=True)    
      

In [505]:
x = df_right['Time'].values[0]+7
x

135.2883

In [506]:
indexNames = df_right[df_right['Time'] <= x].index
# Delete these row indexes from dataFrame
df_right.drop(indexNames , inplace=True)

In [507]:
indexNames =df_right[df_right[' Hand Type'] == 'left'].index
# Delete these row indexes from dataFrame
df_right.drop(indexNames , inplace=True)

In [508]:
indexNames =df_right[df_right[' # hands'] == 2].index
# Delete these row indexes from dataFrame
df_right.drop(indexNames , inplace=True)

In [509]:
df_right=df_right.dropna()

In [510]:
df_right

,Time,Frame ID,Hand Type,# hands,Position X,Position Y,Position Z,Velocity X,Velocity Y,Velocity Z,...,Yaw,Wrist Pos X,Wrist Pos Y,Wrist Pos Z,Elbow pos X,Elbow Pos Y,Elbow Pos Z,Grab Strenth,Grab Angle,Pinch Strength
421,135.3024,16287,right,1,11.930320,199.9861,4.981665,-35.512210,-165.34660,-57.68153,...,-0.906612,75.457400,206.0521,45.38764,284.3946,140.01670,201.9318,0.0,0.561324,0.0
422,135.3194,16289,right,1,11.191900,195.3075,3.661716,-43.587440,-309.83740,-77.18515,...,-0.904311,74.461410,202.4562,44.28679,282.6278,140.97120,203.6773,0.0,0.626321,0.0
423,135.3357,16291,right,1,10.533080,189.0297,2.829388,-30.403600,-363.00330,-43.79815,...,-0.901862,73.600600,197.4387,43.54082,280.6384,142.54010,206.7526,0.0,0.636596,0.0
424,135.3524,16293,right,1,10.243000,181.9816,2.064330,-15.424000,-423.21200,-43.99948,...,-0.901268,73.195370,191.7202,42.73155,279.7357,148.13820,209.9330,0.0,0.701644,0.0
425,135.3690,16295,right,1,10.020610,174.8592,1.228467,-8.733602,-413.45130,-53.88054,...,-0.897777,72.686710,186.1613,41.98006,278.6890,152.52150,212.1112,0.0,0.786548,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4389,201.0806,23867,right,1,-8.623579,163.7605,207.301100,-271.901400,-12.95597,1133.33900,...,0.004875,5.541947,119.3280,267.78980,171.7839,-42.17188,410.2199,0.0,0.765803,0.0
4390,201.0967,23869,right,1,-13.097740,162.5545,225.698500,-266.212900,-102.12860,1029.19300,...,0.003377,1.698353,117.0805,285.09920,179.2368,-44.11964,413.5244,0.0,0.783391,0.0
4391,201.1133,23871,right,1,-16.722380,160.9650,240.584400,-192.594300,-87.53856,769.65150,...,0.004909,-1.484266,114.0260,298.58760,186.0559,-44.35751,415.7693,0.0,0.684677,0.0
4392,201.1299,23873,right,1,-19.684070,160.7849,251.099800,-158.997200,73.45740,557.20370,...,0.004470,-4.015731,112.9914,308.15980,190.9921,-42.38844,416.7549,0.0,0.591905,0.0


In [511]:
df_right_copy=df_right